# Detecting temperature targets
##### authors: Elizabeth A. Barnes and Noah Diffenbaugh
##### version: v0.1.0

## Python setup stuff

The following code is for setting up a local environment.
```
conda create --name env-noah python=3.9
conda activate env-noah
pip install tensorflow==2.7.0
pip install tensorflow-probability==0.15.0
pip install --upgrade numpy scipy pandas statsmodels matplotlib seaborn palettable progressbar2 tabulate icecream flake8 keras-tuner sklearn jupyterlab black isort jupyterlab_code_formatter
pip install -U scikit-learn
pip install silence-tensorflow tqdm
conda install -c conda-forge cmocean cartopy
conda install -c conda-forge xarray dask netCDF4 bottleneck
conda install -c conda-forge nc-time-axis
```

Use the command
```python -m pip freeze > requirements.txt```
to make a pip installation list.

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
print('IN_COLAB = ' + str(IN_COLAB))

In [ ]:
if IN_COLAB:
    !pip install xarray==0.20.2
    !pip install nc-time-axis

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Softmax

import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
savefig_dpi = 300
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")  
print(f"tensorflow version = {tf.__version__}")  

## Define parameters

In [ ]:
TARGET_TEMP = 1.5
BASELINE_YEARS = ('1850','1899')

## Prepare the data

In [ ]:
if IN_COLAB:
    !pip install wget  
    import wget
    nc_filename = wget.download('https://eabarnes-data.atmos.colostate.edu/share/b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.annual.nc')
else:
    DATA_DIRECTORY = 'data/'
    nc_filename = DATA_DIRECTORY + 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.annual.nc'

da = xr.open_dataarray(nc_filename)
da

## Set training / validation / testing

In [ ]:
TRAIN_MEMBERS = np.arange(0,24)
VAL_MEMBERS   = np.arange(24,27)
TEST_MEMBERS  = np.arange(27,30)
print(TRAIN_MEMBERS, VAL_MEMBERS, TEST_MEMBERS)

data_train = da[TRAIN_MEMBERS,:,:]
data_val   = da[VAL_MEMBERS,:,:,:]
data_test  = da[TEST_MEMBERS,:,:,:]

## Define the target year and labels

For starters, we will define the target year as the year that the ensemble mean, global mean temperature crosses the ```TARGET_TEMP``` threshold.

In [ ]:
# compute the ensemble mean, global mean temperature
# these computations should be based on the training set only

da_ens = data_train.mean(axis=0)
weights = np.cos(np.deg2rad(da_ens.lat))
weights.name = "weights"
temp_weighted = da_ens.weighted(weights)
global_mean = temp_weighted.mean(("lon", "lat"))


# compute the target year 
baseline_mean = global_mean.sel(time=slice(BASELINE_YEARS[0],BASELINE_YEARS[1])).mean('time')
iwarmer = np.where(global_mean.values > baseline_mean.values+TARGET_TEMP)[0]
target_year = global_mean["time"].values[iwarmer[0]]


# plot the calculation to make sure things make sense
global_mean.plot(linewidth=2,label='data',color="tab:blue")
plt.axhline(y=baseline_mean, color='gray', linestyle='-', label='basline temp')
plt.axhline(y=baseline_mean+TARGET_TEMP, color='orange', linestyle='-', label='target temp')
plt.axvline(x=target_year,color='red',linewidth=2, linestyle='--', label='target year')
plt.xlabel('year')
plt.ylabel('temp (K)')
plt.title('ensemble mean, global mean temperature\ntargets [' + str(target_year.year) + ', ' + str(TARGET_TEMP) + 'C]')
plt.legend()
plt.show()


In [ ]:
# define the labels
TARGET_YEAR = target_year.year
print('TARGET_YEAR = ' + str(TARGET_YEAR))

labels = da['time.year'].values
labels = TARGET_YEAR - labels
print('labels = ' + str(labels))

## Setup the network

In [ ]:
# define the model
def compile_model(x_train):

    # First we start with an input layer
    inputs = Input(shape=x_train.shape[1:]) 

    normalizer = tf.keras.layers.Normalization()
    normalizer.adapt(x_train)
    layers = normalizer(inputs)

    layers = Dropout(rate=.0,seed=SEED)(layers) # if Dropout(0), no dropout is applied
    
    for hidden, activation in zip(HIDDENS, ACTIVATIONS):
        layers = Dense(hidden, activation=activation,
                       kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.00, l2=RIDGE),
                       bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
                       kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)


    # Output layer has a softmax function to convert output to class likelihood
    output_layer = Dense(1, activation='linear',
                      bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
                      kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)

    # Constructing the model
    model = Model(inputs, output_layer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE) # Using the Adam optimizer
    model.compile(optimizer=optimizer, loss=LOSS, metrics=['mse',])

    model.summary()
    
    return model

# Early Stopping
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                   patience=10,
                                                   verbose=1,
                                                   mode='auto',
                                                   restore_best_weights=True)

## Train the network

In [ ]:
x_train = data_train.values.reshape((data_train.shape[0]*data_train.shape[1],data_train.shape[2]*data_train.shape[3]))
x_val   = data_val.values.reshape((data_val.shape[0]*data_val.shape[1],data_val.shape[2]*data_val.shape[3]))
x_test  = data_test.values.reshape((data_test.shape[0]*data_test.shape[1],data_test.shape[2]*data_test.shape[3]))

y_train = np.tile(labels,data_train.shape[0])
y_val   = np.tile(labels,data_val.shape[0])
y_test  = np.tile(labels,data_test.shape[0])

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

In [ ]:
SEED = 8889
LEARNING_RATE = 0.0001
BATCH_SIZE = 64
NUM_EPOCHS = 500
HIDDENS = [10,10]
ACTIVATIONS = ['relu'] * len(HIDDENS) 
LOSS = 'mae'
RIDGE = .1
VERBOSITY = 0
#----------------------------------------

tf.keras.backend.clear_session()            
tf.random.set_seed(SEED)
np.random.seed(SEED)

model = compile_model(x_train)
history = model.fit(x_train, y_train, 
                    epochs=NUM_EPOCHS, 
                    verbose=VERBOSITY,
                    batch_size = BATCH_SIZE, 
                    shuffle=True,
                    validation_data=[x_val, y_val],
                    callbacks=[EARLY_STOPPING],
                   )
#----------------------------------------
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
x_plot = x_test #x_test #x_val
y_plot = y_test #y_test #y_val
years = np.arange(1850,2101)

predict_plot = model.predict(x_plot)
mae = np.mean(np.abs(predict_plot[:,0]-y_plot[:]))

#--------------------------------
plt.figure(figsize=(7,5))
plt.plot(TARGET_YEAR-y_plot, predict_plot,'.')

plt.axvline(x=TARGET_YEAR, linestyle='-', color='gray')
plt.axhline(y=0,color='gray')

plt.plot(years,labels,'--',color='fuchsia', linewidth=3, label='truth')

plt.legend()
plt.xlabel('year of map')
plt.ylabel('predicted number of years until target is reached')
plt.title('Testing Data\ntargets [' + str(TARGET_YEAR) + ', ' + str(TARGET_TEMP) + 'C]; MAE = ' + str(np.round(mae,2)) + ' years')

if IN_COLAB==False:
    plt.savefig('figures/initial_result_seed' + str(SEED) + '.png', dpi=savefig_dpi)
plt.show()